In [1]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

In [ ]:
# GitHub API 토큰 설정 (GitHub에서 Personal Access Token 생성 필요)
GITHUB_TOKEN = 'your_github_token_here'  # 여기에 토큰 입력
HEADERS = {'Authorization': f'token {GITHUB_TOKEN}'}

# 검색 쿼리 설정
QUERY = 'robot'

# GitHub API 엔드포인트
SEARCH_URL = 'https://api.github.com/search/repositories'